In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import codecs

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
with codecs.open('./data/eng-fra.txt', encoding='utf-8', mode='rb') as f:
    data = f.read()
    
line = data.strip().split('\n')

In [3]:
line

['Go.\tVa !',
 'Run!\tCours\u202f!',
 'Run!\tCourez\u202f!',
 'Wow!\tÇa alors\u202f!',
 'Fire!\tAu feu !',
 "Help!\tÀ l'aide\u202f!",
 'Jump.\tSaute.',
 'Stop!\tÇa suffit\u202f!',
 'Stop!\tStop\u202f!',
 'Stop!\tArrête-toi !',
 'Wait!\tAttends !',
 'Wait!\tAttendez !',
 'I see.\tJe comprends.',
 "I try.\tJ'essaye.",
 "I won!\tJ'ai gagné !",
 "I won!\tJe l'ai emporté !",
 'Oh no!\tOh non !',
 'Attack!\tAttaque !',
 'Attack!\tAttaquez !',
 'Cheers!\tSanté !',
 'Cheers!\tÀ votre santé !',
 'Cheers!\tMerci !',
 'Get up.\tLève-toi.',
 "Got it!\tJ'ai pigé !",
 'Got it!\tCompris !',
 'Got it?\tPigé\u202f?',
 'Got it?\tCompris\u202f?',
 "Got it?\tT'as capté\u202f?",
 'Hop in.\tMonte.',
 'Hop in.\tMontez.',
 'Hug me.\tSerre-moi dans tes bras !',
 'Hug me.\tSerrez-moi dans vos bras !',
 'I fell.\tJe suis tombée.',
 'I fell.\tJe suis tombé.',
 'I know.\tJe sais.',
 'I left.\tJe suis parti.',
 'I left.\tJe suis partie.',
 "I lost.\tJ'ai perdu.",
 "I'm 19.\tJ'ai 19 ans.",
 "I'm OK.\tJe vais bien.",

In [4]:
line[0].split('\t')

['Go.', 'Va !']

In [32]:
en = []
fra = []

def preprocess_word(s):
    
    s = re.sub(r'([.,?!])', r' \1', s)
    s = re.sub(r'[^a-zA-Z.,!?]+', r' ', s)\
    
    return s

for l in line:
    pair = l.lower().split('\t')
    en_word = pair[0]
    fra_word = pair[1]
    
    en.append(preprocess_word(pair[0]))
    fra.append(preprocess_word(pair[1]))

In [22]:
en[:5]

['go .', 'run !', 'run !', 'wow !', 'fire !']

In [23]:
fra[:5]

['va !', 'cours !', 'courez !', ' a alors !', 'au feu !']

In [33]:
def build_dict(eng_list, fra_list):
    
    start_token = 0
    end_token = 1
    
    en_idx = {}
    fra_idx = {}
    idx_en = {}
    idx_fra = {}
    
    en_idx['SOS'] = start_token
    en_idx['EOS'] = end_token
    fra_idx['SOS'] = start_token
    fra_idx['EOS'] = end_token
    
    idx = 2
    for e in eng_list:
        for word in e.split():
            if word not in en_idx:
                en_idx[word] = idx
                idx += 1
    idx = 2
    for f in fra_list:
        for word in f.split():
            if word not in fra_idx:
                fra_idx[word] = idx
                idx += 1
            
    idx_en[0] = 'SOS'
    idx_en[1] = 'EOS'
    idx_fra[0] = 'SOS'
    idx_fra[1] = 'EOS'
        
    for word, idx in en_idx.items():
        idx_en[idx] = word
    for word, idx in fra_idx.items():
        idx_fra[idx] = word
    
    return en_idx, fra_idx, idx_en, idx_fra

In [34]:
en_idx, fra_idx, idx_en, idx_fra = build_dict(en, fra)